In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
data=pd.read_csv("/kaggle/input/nba-salaries-prediction-in-20172018-season/2017-18_NBA_salary.csv")
data.head()

In [ ]:
data.sort_values("Salary",ascending=False)[["Player","Salary","PER"]][0:10]

Gordon Hayward is injured. Mike Conley is odd man out. 

In [ ]:
data[["PER","Salary"]].corr()

Interesting. Per and Salary don't have strong correlation.

In [ ]:
data[["PER","Salary","Age"]].corr()

Salary and Age. Better Correlation.

In [ ]:
data[["WS","Age"]].corr()

It is also interesting.

In [ ]:
data[["DWS","Age"]].corr()

In [ ]:
data[["USG%","Salary"]].corr()

Usage Rate.

In [ ]:
data.sort_values("USG%",ascending=False)[["Player","Salary","USG%"]][0:10]

Rookie contracts and bench warmers are anomaly. We should Delete this.

In [ ]:
data2=data[data["USG%"]<=37]

In [ ]:
data2[["USG%","Salary"]].corr()

Also we should delete rarely used players.

In [ ]:
data2.sort_values("MP",ascending=True)[["Player","Salary","MP"]][0:20]

In [ ]:
data3=data2[data2["MP"]>=200]

In [ ]:
data3[["USG%","Salary"]].corr()

In [ ]:
data3[["PER","Salary"]].corr()

Okay Now Dataset is better. 

In [ ]:
data3.sort_values("Salary",ascending=False)[["Player","Salary","PER"]][0:20]

Carmelo Anthony is also anomaly. 

In [ ]:
data4=data3[data3["Player"]!="Carmelo Anthony"]

In [ ]:
data4[["PER","Salary"]].corr()

Rookie Scale contract also a problem but not have practical way the eliminate them.

In [ ]:
data4.sort_values("OBPM",ascending=False)[["Player","Salary","OBPM"]][0:10]

Trey Burke is a bench warmer. 

In [ ]:
data4[data4["Player"]=="Trey Burke"]

Okay Play at least 41 game (play half of the regular season). 

In [ ]:
data5=data4[data4["G"]>=41]

In [ ]:
data5.sort_values("OBPM",ascending=False)[["Player","Salary","OBPM"]][0:10]

In [ ]:
data5[["PER","Salary"]].corr()

In [ ]:
data5.columns

Okay we should create minute Per Game column.

In [ ]:
data5["MinPerGame"]=data5["MP"]/data5["G"]

In [ ]:
data5[["MinPerGame","Salary"]].corr()

MinPerGame col work just fine

In [ ]:
data5.sort_values("MinPerGame",ascending=False)[["Player","Salary","MinPerGame"]][0:10]

Andrew Wiggins Rookie Scale contract is problem but It's NBA's normal. 

In [ ]:
data5.sort_values("MinPerGame",ascending=True)[["Player","Salary","MinPerGame"]][0:10]

Okay Data is look ready. 

In [ ]:
data5.head()
y=data5.iloc[:,1:2]
x=data5.iloc[:,3:]
del x["Tm"]

In [ ]:
from sklearn.feature_selection import SelectKBest,f_regression
x_new = SelectKBest(f_regression, k=10).fit_transform(x, y)

In [ ]:
x_new.shape

Okay Decision Tree Regressor is my first choice.

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(x_new, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtreg=DecisionTreeRegressor()
dtreg.fit(x_train,y_train)
ypred=dtreg.predict(x_test)

import sklearn.metrics as metrik
metrik.mean_absolute_error(y_pred=ypred,y_true=y_test)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=7)
knn.fit(x_train,y_train)
ypred=knn.predict(x_test)
metrik.mean_absolute_error(y_pred=ypred,y_true=y_test)

Work better than Decision Tree.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=int(math.sqrt(x_train.shape[0])/2))
knn.fit(x_train,y_train)
ypred=knn.predict(x_test)
metrik.mean_absolute_error(y_pred=ypred,y_true=y_test)

Okay 7 is the ideal K.

In [ ]:
from tpot import TPOTRegressor
tpot = TPOTRegressor(verbosity=2, random_state=19,max_time_mins=67)
tpot.fit(x_train, y_train)
ypred=tpot.predict(x_test)
metrik.mean_absolute_error(y_pred=ypred,y_true=y_test)

In [ ]:
tpot.export("pipeline.py")

It's worked.